In [ ]:
from falsevisir import *
from pprint import pprint

pprint(CFG)

CFG['preprocess_images']['blur_sigma'] = 1


CFG['downsize'] = 400

CFG['extract_features']['patch_size'] = CFG['downsize'] // 8

pprint(CFG)

im_paths = "samples/vis_samples/a002_palette.jpg", "samples/ir_samples/a002_s002__TL.jpg"
vi_path, ir_path = [Path(fp) for fp in im_paths]

#%% Load images
vi_image, ir_image = [load_image(fp) for fp in (vi_path, ir_path)]

print(info(vi_image))
print(info(ir_image))
#show_images((vi_image, ir_image))

#%% Resize to same height
vi_image, ir_image = resize_images((vi_image, ir_image))

print(info(vi_image))
print(info(ir_image))
show_images((vi_image, ir_image))

In [ ]:
CFG['preprocess_images']['edge'] = 1

#%% Warp images
vi_image_w, ir_image_w = warp_images(vi_image, ir_image, show=True)

In [ ]:
#%% Blend images
blend_im = blend_image(vi_image_w, ir_image_w, weight=.5)
print(info(blend_im))

#%% False color image
false_im = false_image(vi_image_w, ir_image_w)
print(info(false_im))

#%% Show results
show_images((vi_image_w, ir_image_w, blend_im, false_im), labels=("VIS", "IR", "BLEND", "FALSE_COLOR"))

In [ ]:
#%% Save results

dst_dir = vi_path.parent / f"false_color_images"
dst_dir.mkdir(exist_ok=True)

# Warped images
save_image(dst_dir / f"{ir_path.stem}_{vi_path.stem}_vi_warp.png", vi_image_w)
save_image(dst_dir / f"{ir_path.stem}_{vi_path.stem}_ir_warp.png", ir_image_w)

# Blended images
save_image(dst_dir / f"{ir_path.stem}_{vi_path.stem}_blend.png", blend_im)
save_image(dst_dir / f"{ir_path.stem}_{vi_path.stem}_falsecolor.png", false_im)



In [ ]:
for ch in range(3):
    save_image(dst_dir / f"{ir_path.stem}_{vi_path.stem}_vi_warp_{ch}.png", vi_image_w[:,:,ch])